In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence = True

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
outdir_csv = debris_prms.mb_binned_fp
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)

In [3]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])
# dc_shp_subset

rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi_subset

209 glaciers in region 16 are included in this model run: ['00080', '00141', '00163', '00173', '00176', '00177', '00205', '00213', '00214', '00216', '00228', '00244', '00248', '00256', '00261', '00274', '00285', '00287', '00288', '00289', '00299', '00331', '00332', '00337', '00360', '00361', '00362', '00363', '00366', '00368', '00370', '00372', '00373', '00410', '00413', '00417', '00427', '00428', '00433', '00485', '00486', '00493', '00496', '00500', '00516', '00540', '00543', '00560', '00566', '00582'] and more
This study is focusing on 209 glaciers in region [16]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,79,RGI60-16.00080,-70.8085,-15.59680,16,1,2.350,5015,5301,5130,11.0,166,1592,0,0,9,20019999,80,16.00080,16.00080
1,140,RGI60-16.00141,-70.7501,-15.30790,16,1,2.558,4944,5343,5165,18.8,194,1426,0,0,9,20019999,141,16.00141,16.00141
2,162,RGI60-16.00163,-70.6962,-15.34450,16,1,2.305,4988,5283,5155,15.5,156,1456,0,0,9,20019999,163,16.00163,16.00163
3,172,RGI60-16.00173,-70.7102,-15.37590,16,1,2.115,4952,5370,5176,19.3,201,1376,0,0,9,20019999,173,16.00173,16.00173
4,175,RGI60-16.00176,-70.7199,-15.35290,16,1,2.546,4987,5380,5185,16.2,231,1583,0,0,9,20019999,176,16.00176,16.00176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2490,RGI60-16.02497,-77.6912,-8.90350,16,1,3.398,4638,5888,5191,31.6,239,2942,0,0,9,20031018,2497,16.02497,16.02497
205,2511,RGI60-16.02518,-77.7162,-8.88227,16,1,3.588,4637,6165,5196,33.4,259,2611,0,0,9,20031018,2518,16.02518,16.02518
206,2559,RGI60-16.02566,-77.6638,-8.89992,16,1,2.068,4639,5990,5143,36.2,184,3028,0,0,9,20031018,2566,16.02566,16.02566


In [4]:
if os.path.exists(debris_prms.dhdt_vel_fns_fp) == False:
    os.makedirs(debris_prms.dhdt_vel_fns_fp)
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '11.03005')

In [6]:
# ===== PROCESS EACH GLACIER =====
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str

    if os.path.exists(debris_prms.hd_fp + debris_prms.hd_fn_sample.replace('XXXX',glac_str_noleadzero)) == False:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()


            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            # Debris cover
                            dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                            gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dc_dhdt.mask = gf.dc_mask
                                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dc_dhdt_pixels = len(gf.dc_dhdt.nonzero()[0])
                                if dc_dhdt_pixels / gf.dc_area.count() * 100 > 75:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 dc pixels:', gf.dc_area.count())
                                        print('# dc_dhdt_pixels:', dc_dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        gf.dc_vm = gf.vm.copy()
                        gf.dc_vm.mask = gf.dc_mask
                        # Count velocity pixels
                        dc_vel_pixels = len(gf.dc_vm.nonzero()[0])
                        if debug:
                                print('\n# z1 dc pixels:', gf.dc_area.count())
                                print('# dc vel_pixels:', dc_vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if dc_vel_pixels / gf.dc_area.count() * 100 > 75:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, vx_fn=vx_fn_wglacier, 
                              gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, calc_emergence=True, 
                              debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
    #             plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)

# Save updated filenames
dhdt_vel_fns_df.to_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 0 RGI60-16.00080


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


1 1 RGI60-16.00141
2 2 RGI60-16.00163
3 3 RGI60-16.00173
4 4 RGI60-16.00176
5 5 RGI60-16.00177
6 6 RGI60-16.00205
7 7 RGI60-16.00213
8 8 RGI60-16.00214
9 9 RGI60-16.00216
10 10 RGI60-16.00228
11 11 RGI60-16.00244
12 12 RGI60-16.00248
13 13 RGI60-16.00256
14 14 RGI60-16.00261
15 15 RGI60-16.00274
16 16 RGI60-16.00285
17 17 RGI60-16.00287
18 18 RGI60-16.00288
19 19 RGI60-16.00289
20 20 RGI60-16.00299
21 21 RGI60-16.00331
22 22 RGI60-16.00332
23 23 RGI60-16.00337
24 24 RGI60-16.00360
25 25 RGI60-16.00361
26 26 RGI60-16.00362
27 27 RGI60-16.00363
28 28 RGI60-16.00366
29 29 RGI60-16.00368
30 30 RGI60-16.00370
31 31 RGI60-16.00372
32 32 RGI60-16.00373
33 33 RGI60-16.00410
34 34 RGI60-16.00413
35 35 RGI60-16.00417
36 36 RGI60-16.00427
37 37 RGI60-16.00428
38 38 RGI60-16.00433
39 39 RGI60-16.00485
40 40 RGI60-16.00486
41 41 RGI60-16.00493
42 42 RGI60-16.00496
43 43 RGI60-16.00500
44 44 RGI60-16.00516
45 45 RGI60-16.00540
46 46 RGI60-16.00543
47 47 RGI60-16.00560
48 48 RGI60-16.00566
49 49 RGI6

In [7]:
print('\n\nDONE!\n\n')



DONE!




In [ ]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)